# 환경 설정

In [1]:
# 환경변수 불러오기
from dotenv import load_dotenv
load_dotenv()

from huggingface_hub import login
import os
login(token=os.getenv("HUGGINGFACE_API_KEY"))

In [2]:
import torch 
print(torch.__version__)

device = "cuda:1" if torch.cuda.is_available() else "cpu"
print(device)

2.8.0+cu126
cuda:1


# 기존 사전학습 모델 체험하기

## 1. 모델 불러오기

In [3]:
from transformers import BitsAndBytesConfig
import torch

# 8비트(LoRA + int8) —— 정확도 보존↑, 메모리 절약은 중간(≈2~3배)
## VRAM 12GB에서 안정/보수적으로 돌리고 싶을 때 기본 선택
## 추론 품질 손실을 최소화하고 싶을 때도 선호
quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,          # ✅ 가중치를 8bit로 로드(메모리 절감, 정확도 손실 적음)
    load_in_4bit=False,         # 4bit 경로 비활성화(헷갈림 방지)

    # llm_int8_threshold:
    #  - outlier(이상치) 채널을 감지해 해당 채널만 고정밀 경로로 우회하는 기준.
    #  - 기본 6.0이 널리 사용. 숫자를 낮추면 우회 채널이 늘어 정밀↑(메모리/속도↓).
    llm_int8_threshold=6.0,

    # llm_int8_has_fp16_weight:
    #  - 일부 레이어(예: outlier가 많은 레이어)의 가중치를 fp16로 유지할지 여부.
    #  - False면 전반적으로 8bit 경로에 머무름(메모리↓). True면 더 보수적(정확도↑, 메모리↑).
    llm_int8_has_fp16_weight=False,
)

# 4비트(QLoRA) —— 메모리 절약 극대화(≈4~5배), 대형 모델/대량 배치에 유리
## 4070 12GB에서 최대한 큰 모델/배치를 돌리고 싶을 때 선택
## 약간의 정밀도 손실 감수 가능할 때
# quantization_config = BitsAndBytesConfig(
#     load_in_4bit=True,          # 가중치를 4bit로 로드(QLoRA 핵심, VRAM 절약 극대화)
#     load_in_8bit=False,         # 8bit 경로 비활성화

#     # 연산 정밀도(계산 dtype):
#     # - 가중치는 4bit로 ‘보관’하지만, 실제 곱셈/합산은 16비트로 수행.
#     # - 4070(아다러너)라면 bfloat16 권장(underflow에 강하고 안정적).
#     bnb_4bit_compute_dtype=torch.bfloat16,  # 또는 torch.float16 (환경에 따라 테스트)

#     # Double Quantization:
#     # - 1차 4bit 양자화에 쓰인 스케일/제로포인트 자체를 다시 압축.
#     # - 메모리 추가 절감 효과. 보통 품질 영향은 미미해 ON 권장.
#     bnb_4bit_use_double_quant=True,

#     # 4bit 포맷:
#     # - "nf4": NormalFloat4. LLM 분포에 맞춘 4bit 포맷으로 QLoRA 기본 추천.
#     # - "fp4": Float4. 연구/특수 목적이 아니라면 nf4가 일반적으로 더 좋음.
#     bnb_4bit_quant_type="nf4",
# )

In [37]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = "google/gemma-2-2b-it" # "google/gemma-2b-it"

model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    quantization_config=quantization_config,
    dtype=torch.bfloat16,   # torch_dtype -> dtype
    attn_implementation="eager"
)
# pip install huggingface_hub[hf_xet] 다운로드 속도 높아짐

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## 2. 토크나이저 불러오기

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
print(tokenizer.pad_token, tokenizer.eos_token)

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
# [Tip] 파인튜닝 시 패딩을 안전하기 위한 설정
# 1. tokenizer.pad_token = tokenizer.eos_token
# 패딩 토큰이 되어 있지 않은 경우가 있어 만약 패딩이 필요할 때 오류가 날 수 있음(pad_token=None)
# 2. tokenizer.padding_side = "right"
# 모델이 왼쪽에서 오른쪽으로 단어를 예측하기 때문에, 오른쪽으로 설정하는게 좋음
# 이 설정을 사용하면, 예를 들어 최대 길이가 8인 모델에 "안녕하세요"라는 문장을 입력할 때, 토크나이저는 다음과 같이 처리합니다.
# 원래 토큰 시퀀스: [안, 녕, 하, 세, 요]
# 패딩된 토큰 시퀀스: [안, 녕, 하, 세, 요, <pad>, <pad>, <pad>

<pad> <eos>


### 1) 토크나이저 파라미터 이해하기

In [6]:
print(tokenizer.model_max_length)

1000000000000000019884624838656


In [7]:
print(tokenizer.padding_side)

right


In [10]:
# 패딩 테스트 
# 토큰화는 글자 단위로 쪼개는게 아니라 의미 단위로 쪼개는 것 주의
test = tokenizer(
    ["안녕하세요", "반가워요", "안녕히 계세요"],
    max_length=5,            # 최대 길이를 5로 고정
    truncation=True,         # 길이가 5보다 크면 잘라냄
    padding="max_length",    # 길이가 5보다 짧으면 패딩처리
    padding_side="right"     # 패딩은 오른쪽에 붙임
)

test
# {'input_ids': [[2, 238179, 243415, 204551, 1], [2, 238559, 236361, 239779, 237526], [2, 238179, 243415, 239055, 74715]]
# 반복되는 2는 <bos> / 2381799 243415는 안녕

{'input_ids': [[2, 238179, 243415, 204551, 1], [2, 238559, 236361, 239779, 237526], [2, 238179, 243415, 239055, 74715]], 'attention_mask': [[1, 1, 1, 1, 0], [1, 1, 1, 1, 1], [1, 1, 1, 1, 1]]}

### 2) 토크나이저 활용하기

In [12]:
# 문장의 경계를 알려주는 토큰: <bos>, <eos>
# <bos> (begin of sequence) - 문장 시작
# <eos> (end of sequence) - 문장 끝

In [13]:
text = "안녕하세요"

# 텍스트를 숫자로 토크나이징
tokenized_text = tokenizer(text)
print(tokenized_text)

# 숫자를 텍스트로 디코딩
decoded_text = tokenizer.decode(tokenized_text["input_ids"], skip_special_tokens=False)
print(decoded_text)

{'input_ids': [2, 238179, 243415, 204551], 'attention_mask': [1, 1, 1, 1]}
<bos>안녕하세요


In [14]:
# 대부분의 it 모델들은 system이라는 role을 이해하지 못해 user에 system 정보 추가하는 것으로 추가
# TemplateError: System role not supported
#  {
#       "role": "system",
#       "content": "너는 시스템이야"
#   },
messages = [
    [
        {
            "role": "user",
            "content": "친절하게 답변해주세요\n 안녕하세요"
        },
        {
            "role": "assistant",
            "content": "안녕하세요. 반가워요"
        },
    ],
]

# 대화 구조를 숫자로 토크나이징
tokenized_messages = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=False,
    tokenize=True,
    return_dict=True,
    return_tensors="pt"
)
print(tokenized_messages)

print('-' * 100)

tokenized_messages_str = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=False,
    tokenize=False,
)
print(tokenized_messages_str)

print('-' * 100)

# 숫자를 텍스트로 디코딩
decoded_messages = tokenizer.decode(
    tokenized_messages["input_ids"][0],
    skip_special_tokens=False
)
print(decoded_messages)

{'input_ids': tensor([[     2,    106,   1645,    108, 240471, 240046, 101969, 235248, 241305,
         239042, 237138, 237014,  96673,    108,  70685, 243415, 204551,    107,
            108,    106,   2516,    108, 238179, 243415, 204551, 235265,  78821,
         236361, 239779, 237526,    107,    108]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1]])}
----------------------------------------------------------------------------------------------------
['<bos><start_of_turn>user\n친절하게 답변해주세요\n 안녕하세요<end_of_turn>\n<start_of_turn>model\n안녕하세요. 반가워요<end_of_turn>\n']
----------------------------------------------------------------------------------------------------
<bos><start_of_turn>user
친절하게 답변해주세요
 안녕하세요<end_of_turn>
<start_of_turn>model
안녕하세요. 반가워요<end_of_turn>



In [15]:
tokenizer.decode([240471])

'친'

# 학습 준비

## 1. 학습 데이터

### 1) HuggingFace Dataset 불러오기

In [16]:
from datasets import load_dataset
dataset = load_dataset("daekeun-ml/naver-news-summarization-ko")

In [17]:
dataset

DatasetDict({
    train: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
        num_rows: 22194
    })
    validation: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
        num_rows: 2466
    })
    test: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
        num_rows: 2740
    })
})

In [18]:
sample_dataset = dataset['train'].select(range(10))
sample_dataset

Dataset({
    features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
    num_rows: 10
})

In [19]:
def generate_prompts(example):
    messages = [
        {"role": "user",
            "content": "다음 글을 요약해주세요:\n\n {}".format(example['document'])}, 
        {"role": "assistant",
            "content": "{}".format(example['summary'])}
    ]
    chat_message = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
    example["text"] = chat_message+"<eos>"
    return example

In [20]:
sample_dataset = sample_dataset.map(generate_prompts) # map(빠름!)으로 각 요소에 함수를 적용(keywords라는 key 추가됨)

In [21]:
sample_dataset[0]

{'date': '2022-07-03 17:14:37',
 'category': 'economy',
 'press': 'YTN ',
 'title': '추경호 중기 수출지원 총력 무역금융 40조 확대',
 'document': '앵커 정부가 올해 하반기 우리 경제의 버팀목인 수출 확대를 위해 총력을 기울이기로 했습니다. 특히 수출 중소기업의 물류난 해소를 위해 무역금융 규모를 40조 원 이상 확대하고 물류비 지원과 임시선박 투입 등을 추진하기로 했습니다. 류환홍 기자가 보도합니다. 기자 수출은 최고의 실적을 보였지만 수입액이 급증하면서 올해 상반기 우리나라 무역수지는 역대 최악인 103억 달러 적자를 기록했습니다. 정부가 수출확대에 총력을 기울이기로 한 것은 원자재 가격 상승 등 대외 리스크가 가중되는 상황에서 수출 증가세 지속이야말로 한국경제의 회복을 위한 열쇠라고 본 것입니다. 추경호 경제부총리 겸 기획재정부 장관 정부는 우리 경제의 성장엔진인 수출이 높은 증가세를 지속할 수 있도록 총력을 다하겠습니다. 우선 물류 부담 증가 원자재 가격 상승 등 가중되고 있는 대외 리스크에 대해 적극 대응하겠습니다. 특히 중소기업과 중견기업 수출 지원을 위해 무역금융 규모를 연초 목표보다 40조 원 늘린 301조 원까지 확대하고 물류비 부담을 줄이기 위한 대책도 마련했습니다. 이창양 산업통상자원부 장관 국제 해상운임이 안정될 때까지 월 4척 이상의 임시선박을 지속 투입하는 한편 중소기업 전용 선복 적재 용량 도 현재보다 주당 50TEU 늘려 공급하겠습니다. 하반기에 우리 기업들의 수출 기회를 늘리기 위해 2 500여 개 수출기업을 대상으로 해외 전시회 참가를 지원하는 등 마케팅 지원도 벌이기로 했습니다. 정부는 또 이달 중으로 반도체를 비롯한 첨단 산업 육성 전략을 마련해 수출 증가세를 뒷받침하고 에너지 소비를 줄이기 위한 효율화 방안을 마련해 무역수지 개선에 나서기로 했습니다. YTN 류환홍입니다.',
 'link': 'https://n.news.naver.com/mne

### 2) 커스텀 데이터 만들기

In [22]:
# [목표] 다음 신문을 요약해주세요. 그리고 핵심 키워드 5개를 뽑아주세요.

# it 모델을 파인튜닝하는 이유 - 응답 결과의 일관성을 만들기 위해서(지시 사항을 함께 학습 시키기 위함)
# ex) 신문 기사를 요약하고, 핵심 키워드 5개를 뽑아라

In [23]:
# STEP1. 위에서 불러온 데이터를 가지고 keywords를 뽑아주는 데이터를 만들어야 한다.
from openai import OpenAI
client = OpenAI()
# {'date': '2022-07-03 17:14:37',
#  'category': 'economy',
#  'press': 'YTN ',
#  'title': '추경호 중기 수출지원 총력 무역금융 40조 확대',
#  'document': '앵커 정부가 올해 하반기 우리 경제의 버팀목인 수출 확대를 위해 총력을 기울이기로 했습니다. 특히 수출 중소기업의 물류난 해소를 위해 무역금융 규모를 40조 원 이상 확대하고 물류비 지원과 임시선박 투입 등을 추진하기로 했습니다. 류환홍 기자가 보도합니다. 기자 수출은 최고의 실적을 보였지만 수입액이 급증하면서 올해 상반기 우리나라 무역수지는 역대 최악인 103억 달러 적자를 기록했습니다. 정부가 수출확대에 총력을 기울이기로 한 것은 원자재 가격 상승 등 대외 리스크가 가중되는 상황에서 수출 증가세 지속이야말로 한국경제의 회복을 위한 열쇠라고 본 것입니다. 추경호 경제부총리 겸 기획재정부 장관 정부는 우리 경제의 성장엔진인 수출이 높은 증가세를 지속할 수 있도록 총력을 다하겠습니다. 우선 물류 부담 증가 원자재 가격 상승 등 가중되고 있는 대외 리스크에 대해 적극 대응하겠습니다. 특히 중소기업과 중견기업 수출 지원을 위해 무역금융 규모를 연초 목표보다 40조 원 늘린 301조 원까지 확대하고 물류비 부담을 줄이기 위한 대책도 마련했습니다. 이창양 산업통상자원부 장관 국제 해상운임이 안정될 때까지 월 4척 이상의 임시선박을 지속 투입하는 한편 중소기업 전용 선복 적재 용량 도 현재보다 주당 50TEU 늘려 공급하겠습니다. 하반기에 우리 기업들의 수출 기회를 늘리기 위해 2 500여 개 수출기업을 대상으로 해외 전시회 참가를 지원하는 등 마케팅 지원도 벌이기로 했습니다. 정부는 또 이달 중으로 반도체를 비롯한 첨단 산업 육성 전략을 마련해 수출 증가세를 뒷받침하고 에너지 소비를 줄이기 위한 효율화 방안을 마련해 무역수지 개선에 나서기로 했습니다. YTN 류환홍입니다.',
#  'link': 'https://n.news.naver.com/mnews/article/052/0001759333?sid=101',
#  'summary': '올해 상반기 우리나라 무역수지는 역대 최악인 103억 달러 적자를 기록한 가운데, 정부가 하반기에 우리 경제의 버팀목인 수출 확대를 위해 총력을 기울이기로 결정한 가운데, 특히 수출 중소기업의 물류난 해소를 위해 무역금융 규모를 40조 원 이상 확대하고 물류비 지원과 임시선박 투입 등을 추진하기로 했다.'}
def get_keywords(data): 
    messages = [
        {
            "role": "user",
            "content": f"뉴스에서 핵심 키워드를 예시와 같이 명사로 5개 추출해주세요\n\n[뉴스전문] {data['document']}\n\n[예시] 키워드1, 키워드2, 키워드3, 키워드4, 키워드5"
        }
    ]
    output = client.chat.completions.create(model="gpt-5-nano", messages=messages)
    output = output.choices[0].message.content

    data["keywords"] = output

    return data

In [24]:
sample_dataset[0]

{'date': '2022-07-03 17:14:37',
 'category': 'economy',
 'press': 'YTN ',
 'title': '추경호 중기 수출지원 총력 무역금융 40조 확대',
 'document': '앵커 정부가 올해 하반기 우리 경제의 버팀목인 수출 확대를 위해 총력을 기울이기로 했습니다. 특히 수출 중소기업의 물류난 해소를 위해 무역금융 규모를 40조 원 이상 확대하고 물류비 지원과 임시선박 투입 등을 추진하기로 했습니다. 류환홍 기자가 보도합니다. 기자 수출은 최고의 실적을 보였지만 수입액이 급증하면서 올해 상반기 우리나라 무역수지는 역대 최악인 103억 달러 적자를 기록했습니다. 정부가 수출확대에 총력을 기울이기로 한 것은 원자재 가격 상승 등 대외 리스크가 가중되는 상황에서 수출 증가세 지속이야말로 한국경제의 회복을 위한 열쇠라고 본 것입니다. 추경호 경제부총리 겸 기획재정부 장관 정부는 우리 경제의 성장엔진인 수출이 높은 증가세를 지속할 수 있도록 총력을 다하겠습니다. 우선 물류 부담 증가 원자재 가격 상승 등 가중되고 있는 대외 리스크에 대해 적극 대응하겠습니다. 특히 중소기업과 중견기업 수출 지원을 위해 무역금융 규모를 연초 목표보다 40조 원 늘린 301조 원까지 확대하고 물류비 부담을 줄이기 위한 대책도 마련했습니다. 이창양 산업통상자원부 장관 국제 해상운임이 안정될 때까지 월 4척 이상의 임시선박을 지속 투입하는 한편 중소기업 전용 선복 적재 용량 도 현재보다 주당 50TEU 늘려 공급하겠습니다. 하반기에 우리 기업들의 수출 기회를 늘리기 위해 2 500여 개 수출기업을 대상으로 해외 전시회 참가를 지원하는 등 마케팅 지원도 벌이기로 했습니다. 정부는 또 이달 중으로 반도체를 비롯한 첨단 산업 육성 전략을 마련해 수출 증가세를 뒷받침하고 에너지 소비를 줄이기 위한 효율화 방안을 마련해 무역수지 개선에 나서기로 했습니다. YTN 류환홍입니다.',
 'link': 'https://n.news.naver.com/mne

In [25]:
get_keywords(sample_dataset[0])

{'date': '2022-07-03 17:14:37',
 'category': 'economy',
 'press': 'YTN ',
 'title': '추경호 중기 수출지원 총력 무역금융 40조 확대',
 'document': '앵커 정부가 올해 하반기 우리 경제의 버팀목인 수출 확대를 위해 총력을 기울이기로 했습니다. 특히 수출 중소기업의 물류난 해소를 위해 무역금융 규모를 40조 원 이상 확대하고 물류비 지원과 임시선박 투입 등을 추진하기로 했습니다. 류환홍 기자가 보도합니다. 기자 수출은 최고의 실적을 보였지만 수입액이 급증하면서 올해 상반기 우리나라 무역수지는 역대 최악인 103억 달러 적자를 기록했습니다. 정부가 수출확대에 총력을 기울이기로 한 것은 원자재 가격 상승 등 대외 리스크가 가중되는 상황에서 수출 증가세 지속이야말로 한국경제의 회복을 위한 열쇠라고 본 것입니다. 추경호 경제부총리 겸 기획재정부 장관 정부는 우리 경제의 성장엔진인 수출이 높은 증가세를 지속할 수 있도록 총력을 다하겠습니다. 우선 물류 부담 증가 원자재 가격 상승 등 가중되고 있는 대외 리스크에 대해 적극 대응하겠습니다. 특히 중소기업과 중견기업 수출 지원을 위해 무역금융 규모를 연초 목표보다 40조 원 늘린 301조 원까지 확대하고 물류비 부담을 줄이기 위한 대책도 마련했습니다. 이창양 산업통상자원부 장관 국제 해상운임이 안정될 때까지 월 4척 이상의 임시선박을 지속 투입하는 한편 중소기업 전용 선복 적재 용량 도 현재보다 주당 50TEU 늘려 공급하겠습니다. 하반기에 우리 기업들의 수출 기회를 늘리기 위해 2 500여 개 수출기업을 대상으로 해외 전시회 참가를 지원하는 등 마케팅 지원도 벌이기로 했습니다. 정부는 또 이달 중으로 반도체를 비롯한 첨단 산업 육성 전략을 마련해 수출 증가세를 뒷받침하고 에너지 소비를 줄이기 위한 효율화 방안을 마련해 무역수지 개선에 나서기로 했습니다. YTN 류환홍입니다.',
 'link': 'https://n.news.naver.com/mne

In [26]:
sample_dataset = sample_dataset.map(get_keywords)
sample_dataset

Parameter 'function'=<function get_keywords at 0x0000022BDA905B40> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only shown once. Subsequent hashing failures won't be shown.


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Dataset({
    features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary', 'text', 'keywords'],
    num_rows: 10
})

In [27]:
# 만든 데이터 저장하기 
sample_dataset.save_to_disk("naver_summary_custom_data")

Saving the dataset (0/1 shards):   0%|          | 0/10 [00:00<?, ? examples/s]

In [28]:
# 만든 데이터 로드 
from datasets import load_from_disk
sample_dataset = load_from_disk("naver_summary_custom_data")
print(sample_dataset)

Dataset({
    features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary', 'text', 'keywords'],
    num_rows: 10
})


### 3) 학습 데이터 만들기

In [29]:
# STEP2. document, summary, keywords를 가지고 messages 형태의 데이터를 만든다.
def make_prompt(data):
    messages = [
        {
            "role": "user",
            "content": f"뉴스를 요약해주세요.\n\n 그리고 핵심 키워드를 예시와 같이 명사로 5개 추출해주세요\n\n[뉴스전문] {data['document']}\n\n[예시]키워드: 키워드1, 키워드2, 키워드3, 키워드4, 키워드5"
        },
        {
            "role": "assistant",
            "content": f"[키워드]: {data['keywords']}\n\n[뉴스요약]: {data['summary']}"
        }
    ]

    return {
        "text": tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=False,
            tokenize=False
        ) + "<eos>" # <eos>는 따로 추가해줘야 함
    }

In [30]:
sample_dataset[0]

{'date': '2022-07-03 17:14:37',
 'category': 'economy',
 'press': 'YTN ',
 'title': '추경호 중기 수출지원 총력 무역금융 40조 확대',
 'document': '앵커 정부가 올해 하반기 우리 경제의 버팀목인 수출 확대를 위해 총력을 기울이기로 했습니다. 특히 수출 중소기업의 물류난 해소를 위해 무역금융 규모를 40조 원 이상 확대하고 물류비 지원과 임시선박 투입 등을 추진하기로 했습니다. 류환홍 기자가 보도합니다. 기자 수출은 최고의 실적을 보였지만 수입액이 급증하면서 올해 상반기 우리나라 무역수지는 역대 최악인 103억 달러 적자를 기록했습니다. 정부가 수출확대에 총력을 기울이기로 한 것은 원자재 가격 상승 등 대외 리스크가 가중되는 상황에서 수출 증가세 지속이야말로 한국경제의 회복을 위한 열쇠라고 본 것입니다. 추경호 경제부총리 겸 기획재정부 장관 정부는 우리 경제의 성장엔진인 수출이 높은 증가세를 지속할 수 있도록 총력을 다하겠습니다. 우선 물류 부담 증가 원자재 가격 상승 등 가중되고 있는 대외 리스크에 대해 적극 대응하겠습니다. 특히 중소기업과 중견기업 수출 지원을 위해 무역금융 규모를 연초 목표보다 40조 원 늘린 301조 원까지 확대하고 물류비 부담을 줄이기 위한 대책도 마련했습니다. 이창양 산업통상자원부 장관 국제 해상운임이 안정될 때까지 월 4척 이상의 임시선박을 지속 투입하는 한편 중소기업 전용 선복 적재 용량 도 현재보다 주당 50TEU 늘려 공급하겠습니다. 하반기에 우리 기업들의 수출 기회를 늘리기 위해 2 500여 개 수출기업을 대상으로 해외 전시회 참가를 지원하는 등 마케팅 지원도 벌이기로 했습니다. 정부는 또 이달 중으로 반도체를 비롯한 첨단 산업 육성 전략을 마련해 수출 증가세를 뒷받침하고 에너지 소비를 줄이기 위한 효율화 방안을 마련해 무역수지 개선에 나서기로 했습니다. YTN 류환홍입니다.',
 'link': 'https://n.news.naver.com/mne

In [31]:
make_prompt(sample_dataset[0])

{'text': '<bos><start_of_turn>user\n뉴스를 요약해주세요.\n\n 그리고 핵심 키워드를 예시와 같이 명사로 5개 추출해주세요\n\n[뉴스전문] 앵커 정부가 올해 하반기 우리 경제의 버팀목인 수출 확대를 위해 총력을 기울이기로 했습니다. 특히 수출 중소기업의 물류난 해소를 위해 무역금융 규모를 40조 원 이상 확대하고 물류비 지원과 임시선박 투입 등을 추진하기로 했습니다. 류환홍 기자가 보도합니다. 기자 수출은 최고의 실적을 보였지만 수입액이 급증하면서 올해 상반기 우리나라 무역수지는 역대 최악인 103억 달러 적자를 기록했습니다. 정부가 수출확대에 총력을 기울이기로 한 것은 원자재 가격 상승 등 대외 리스크가 가중되는 상황에서 수출 증가세 지속이야말로 한국경제의 회복을 위한 열쇠라고 본 것입니다. 추경호 경제부총리 겸 기획재정부 장관 정부는 우리 경제의 성장엔진인 수출이 높은 증가세를 지속할 수 있도록 총력을 다하겠습니다. 우선 물류 부담 증가 원자재 가격 상승 등 가중되고 있는 대외 리스크에 대해 적극 대응하겠습니다. 특히 중소기업과 중견기업 수출 지원을 위해 무역금융 규모를 연초 목표보다 40조 원 늘린 301조 원까지 확대하고 물류비 부담을 줄이기 위한 대책도 마련했습니다. 이창양 산업통상자원부 장관 국제 해상운임이 안정될 때까지 월 4척 이상의 임시선박을 지속 투입하는 한편 중소기업 전용 선복 적재 용량 도 현재보다 주당 50TEU 늘려 공급하겠습니다. 하반기에 우리 기업들의 수출 기회를 늘리기 위해 2 500여 개 수출기업을 대상으로 해외 전시회 참가를 지원하는 등 마케팅 지원도 벌이기로 했습니다. 정부는 또 이달 중으로 반도체를 비롯한 첨단 산업 육성 전략을 마련해 수출 증가세를 뒷받침하고 에너지 소비를 줄이기 위한 효율화 방안을 마련해 무역수지 개선에 나서기로 했습니다. YTN 류환홍입니다.\n\n[예시]키워드: 키워드1, 키워드2, 키워드3, 키워드4, 키워드5<end_of_turn>\n<start_of_turn>mod

In [32]:
sample_dataset = sample_dataset.map(make_prompt)
sample_dataset

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Dataset({
    features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary', 'text', 'keywords'],
    num_rows: 10
})

In [33]:
sample_dataset[0]["text"]

'<bos><start_of_turn>user\n뉴스를 요약해주세요.\n\n 그리고 핵심 키워드를 예시와 같이 명사로 5개 추출해주세요\n\n[뉴스전문] 앵커 정부가 올해 하반기 우리 경제의 버팀목인 수출 확대를 위해 총력을 기울이기로 했습니다. 특히 수출 중소기업의 물류난 해소를 위해 무역금융 규모를 40조 원 이상 확대하고 물류비 지원과 임시선박 투입 등을 추진하기로 했습니다. 류환홍 기자가 보도합니다. 기자 수출은 최고의 실적을 보였지만 수입액이 급증하면서 올해 상반기 우리나라 무역수지는 역대 최악인 103억 달러 적자를 기록했습니다. 정부가 수출확대에 총력을 기울이기로 한 것은 원자재 가격 상승 등 대외 리스크가 가중되는 상황에서 수출 증가세 지속이야말로 한국경제의 회복을 위한 열쇠라고 본 것입니다. 추경호 경제부총리 겸 기획재정부 장관 정부는 우리 경제의 성장엔진인 수출이 높은 증가세를 지속할 수 있도록 총력을 다하겠습니다. 우선 물류 부담 증가 원자재 가격 상승 등 가중되고 있는 대외 리스크에 대해 적극 대응하겠습니다. 특히 중소기업과 중견기업 수출 지원을 위해 무역금융 규모를 연초 목표보다 40조 원 늘린 301조 원까지 확대하고 물류비 부담을 줄이기 위한 대책도 마련했습니다. 이창양 산업통상자원부 장관 국제 해상운임이 안정될 때까지 월 4척 이상의 임시선박을 지속 투입하는 한편 중소기업 전용 선복 적재 용량 도 현재보다 주당 50TEU 늘려 공급하겠습니다. 하반기에 우리 기업들의 수출 기회를 늘리기 위해 2 500여 개 수출기업을 대상으로 해외 전시회 참가를 지원하는 등 마케팅 지원도 벌이기로 했습니다. 정부는 또 이달 중으로 반도체를 비롯한 첨단 산업 육성 전략을 마련해 수출 증가세를 뒷받침하고 에너지 소비를 줄이기 위한 효율화 방안을 마련해 무역수지 개선에 나서기로 했습니다. YTN 류환홍입니다.\n\n[예시]키워드: 키워드1, 키워드2, 키워드3, 키워드4, 키워드5<end_of_turn>\n<start_of_turn>model\n[키워드]

In [34]:
split_dataset = sample_dataset.train_test_split(test_size=0.2, seed=42)
train_set = split_dataset["train"]
val_set = split_dataset["test"]

## 2. 학습

### 1) 학습 설정

In [35]:
# 전체 가중치를 업데이트 하는 방식 : Full FT
# 가중치 일부만 바꾸는 방식: PEFT(Parameter-Efficient FT) 
## LoRA(저차원행렬의 곱셈) "기존의 가중치는 고정하고 Adapter의 가중치를 업데이트한다."
## uv add peft

from peft import LoraConfig

lora_config = LoraConfig(
    # r: 저차원 랭크(rank). 높일수록 표현력↑, VRAM/연산량↑
    #  - 소형 모델/짧은 데이터: 4~8
    #  - 중형 이상/복잡 과제  : 8~16 (필요시 32)
    r=6,

    # lora_alpha: LoRA 스케일(α). 최종 적용은 (α / r) * (B @ A)
    #  - 너무 크면 과적합/불안정, 너무 작으면 표현력↓
    #  - 보통 r의 2~4배 근처(예: r=8 → α=16~32)
    lora_alpha=8,

    # lora_dropout: 어댑터 경로에만 적용되는 드롭아웃 확률
    #  - 과적합 방지. 데이터 적거나 문체 과적응 막고 싶을 때 0.05~0.1
    #  - 매우 작은 데이터(수십~수백)면 0.0~0.05도 사용
    lora_dropout=0.05,

    # target_modules: LoRA를 적용할 서브모듈 이름 목록
    #  - 주로 Attention의 프로젝션 레이어에 적용
    #  - LLaMA/Gemma 계열: ["q_proj", "k_proj", "v_proj", "o_proj"]가 기본
    #  - 추가로 FFN에 확장하려면 ["up_proj","down_proj","gate_proj"]도 고려
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],

    # bias: 바이어스 파라미터 업데이트 방식
    #  - "none"   : 바이어스는 고정(일반적)
    #  - "lora_only": LoRA에 해당하는 바이어스만 학습
    #  - "all"    : 모든 바이어스 학습(권장 X, 원본 오염 위험)
    bias="none",

    # task_type: 태스크 유형. Causal LM(자연어 생성)에는 "CAUSAL_LM"
    task_type="CAUSAL_LM",

    # modules_to_save: LoRA 외에 함께 학습/저장할 모듈 이름
    #  - 예: "lm_head" 등을 LoRA와 함께 미세 조정하고 싶을 때
    # modules_to_save=["lm_head"],

    # init_lora_weights: LoRA 가중치 초기화 방법
    #  - True(기본): 권장 초기화로 안정적 시작
    #  - False     : 수동 초기화 시
    # init_lora_weights=True,

    # use_rslora: Rank-Stabilized LoRA (표현력 향상 목적)
    #  - True 시 랭크에 안정화 스케일링 적용(큰 r에서 수렴 안정↑)
    # use_rslora=False,

    # inference_mode: 추론 전용 모드로 생성 시 True(학습 시 False)
    # inference_mode=False,

    # layers_to_transform / layers_pattern:
    #  - 특정 레이어 인덱스/패턴에만 LoRA 적용하고 싶을 때 사용
    #  - 예: 마지막 N개 블록만 적용 등
    # layers_to_transform=None,
    # layers_pattern=None,

    # lora_dtype: LoRA 어댑터의 계산 dtype 지정
    #  - None이면 모델 dtype 따름(bfloat16/float16 권장)
    # lora_dtype=None,

    # rank_pattern / alpha_pattern:
    #  - 모듈별로 서로 다른 r/alpha를 주고 싶을 때 딕셔너리로 지정
    #  - 예: {"q_proj": 16, "v_proj": 8} 등 세밀 조정
    # rank_pattern=None,
    # alpha_pattern=None,
)

### 2) 학습

In [36]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="./output",           # 체크포인트/로그 저장 위치 (필수)

    # 학습 길이
    num_train_epochs=1,              # 에폭 수 (기본 3.0). max_steps가 양수면 이 값은 무시됨.

    # 배치/메모리
    per_device_train_batch_size=2,   # 장치당 배치(기본 8). OOM 나면 1로 낮춤
    gradient_accumulation_steps=8,   # 유효배치=2*8=16
    bf16=True,                       # 4070 권장
    gradient_checkpointing=False,    # 메모리 절약(약간 느려짐)

    # 최적화
    learning_rate=2e-4,              # 기본은 5e-5. SFT/LoRA면 조금 높게도 사용
    lr_scheduler_type="cosine",      # 기본은 "linear"
    warmup_ratio=0.0,                # 기본 0.0 → 안정화를 위해 소량 워밍업 권장

    # 평가/저장 (신규 키: eval_strategy)
    # eval_strategy="epoch",           # epoch/steps/no
    save_strategy="epoch",           # epoch/steps/no/best
    save_total_limit=2,              # 오래된 체크포인트 자동 정리

    # 로깅
    logging_strategy="steps",        # 기본 "steps"
    logging_steps=1,                 # 기본 500 → 실습에선 더 자주 보이게
    report_to="none",                # W&B/TensorBoard 안 쓸 때
)

In [38]:
# ------------------------------------------------------------
# TRL의 SFTTrainer: "지시문/대화 데이터"를 가장 간단히 학습하는 Trainer
#  - collator를 따로 만들 필요가 없음(내부에서 처리)
#  - peft_config를 넘기면 LoRA/QLoRA 어댑터만 학습하도록 자동 구성
#  - dataset에 text 열이 있으면 dataset_text_field="text"로 지정
# ------------------------------------------------------------
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,                  # AutoModelForCausalLM 등
    args=args,                    # 학습/로깅/저장은 TrainingArguments에서 통제
    train_dataset=sample_dataset, # Dataset | torch Dataset
    eval_dataset=val_set,       # 선택. 있으면 eval 전략에 따라 평가
    peft_config=lora_config,      # LoRA/QLoRA 적용. (없으면 Full FT)
)

Adding EOS to train dataset:   0%|          | 0/10 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/10 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/10 [00:00<?, ? examples/s]

Adding EOS to eval dataset:   0%|          | 0/2 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/2 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/2 [00:00<?, ? examples/s]

In [ ]:
# 현재 GPU 메모리 캐시 비우기
torch.cuda.empty_cache()

# 학습 시작
# trainer.train() 노트북(+gpu) 하드웨어로는 불가

### 3) 학습된 모델 저장

In [ ]:
# 어댑터 모델 저장
adapter_name = "lora_adapter"

trainer.model.save_pretrained(adapter_name)

In [ ]:
# 기존 모델과 adaptor 모델을 merge하는 것이 필요
from peft import PeftModel
save_model_name = "gemma-3-1b-sum-ko"

# 기존 모델
model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    device_map='auto', 
    torch_dtype=torch.float16
)

# adaptor 모델
model = PeftModel.from_pretrained(
    model, 
    adapter_name, 
    device_map='auto', 
    torch_dtype=torch.float16
)

model = model.merge_and_unload()
model.save_pretrained(save_model_name)

## 3. 추론

In [ ]:
finetune_model = AutoModelForCausalLM.from_pretrained(save_model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
from transformers import pipeline
pipe_finetuned = pipeline(
    "text-generation", 
    model=finetune_model, 
    tokenizer=tokenizer, 
    max_new_tokens=512
)

In [ ]:
doc = dataset['test']['document'][10]

In [ ]:
messages = [
    {
        "role": "user",
        "content": f"뉴스를 요약해주세요.\n\n 그리고 핵심 키워드를 예시와 같이 명사로 5개 추출해주세요\n\n[뉴스전문] {doc}\n\n[예시]키워드: 키워드1, 키워드2, 키워드3, 키워드4, 키워드5"
    }
]
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

In [ ]:
outputs = pipe_finetuned(
    prompt,
    add_special_tokens=True
)
print(outputs[0]["generated_text"][len(prompt):])